In [ ]:
import json
import common
import requests
# Models:
# Mistral, mathstral:7b, To check: falcon3, phi4, llama3m, need to check astrosage


In [ ]:
#sub_cirs = range(21916, 21916+50)
#[21916, 21917,21923, 21924, 21930,21941,22115,22281,22942]
sub_cirs = [39990] #40168
# Good ids: 40168, 40162, 39990, 40409
print([c for c in sub_cirs])


In [ ]:
circulars = common.get_circulars(sub_cirs)
print(circulars[0])

In [ ]:
# prompt = f"""
# Extract the following structured fields from the given astronomy event description:

# 1. Extract **all numeric values** such as:
#    - alpha, fluence, wavelengths, energy, errors, confidence levels, durations, keV values, date-times, coordinates (RA, DEC), radius, redshift (z), etc.
#    - Return them in a dictionary under the key `"numeric_values"` where:
#      - Each key describes the physical quantity (e.g., `"redshift"`, `"wavelengths"`).
#      - If there are multiple values for a quantity, use an array:  
#        Example: `"numeric_values": {{"redshift": [-2, -1], "wavelengths": [200, 3000]}}`

# 2. If any **markdown-style tables** are present, convert them to an array of objects and return under the key `"table"`.

# Return a single **JSON object** with the following top-level keys:
# - `"numeric_values"`
# - `"table"`

# If a section is not applicable or not found, use `null` for that key.

# **Return only the complete JSON object. No explanations.**

# Event Description:
# \"\"\"
# {text}
# \"\"\"
# """

In [ ]:
def getprompt(text):
    prompt = f"""
Extract the following structured fields from the given astronomy event description:

Extract all **relevant numeric values** into a flat JSON object. These include:
- redshift (z), fluence, wavelengths, energy, errors, uncertainties, 
- confidence levels, durations, keV values, date-times, coordinates (RA, DEC), radius, alpha, (any other physics related values) etc.
- Keys must be **clear physical quantities**.
- If multiple values exist for a key, use an array.
- If a value has a unit, store it as an object with `value` and `unit`.
- If a value doesnt exist, use `null`

**Output format Example:**
```json
{{
  "fluence": [{{"value": 2.8e-7, "unit": "erg/cm^2"}}],
  "duration": [1.2],
  "redshift": [1.1],
  "RA": ["03h 18m 00.28s"],
  "DEC": ["-38d 23m 19.1s"]
}}

**Return only the complete JSON object. No explanations.**

Text:
\"\"\"
{text}
\"\"\"
"""
    return prompt

In [ ]:
def getsummaryprompt(text):
    prompt = f"""
Do the following TASKS: 
TASK 1. Rate each sentence (numerically between 1-(total number of sentences) from the given astronomy event description for their importance in terms of scientific inference.
Focus more on numeric measurements, not on date time or authors or collaborators. 
Provide ONLY the orginal sentence and its rating, nothing else.
Format for TASK 1 : [{{"sentence":<sentence>, "rating": <rating>}}]
TASK 2. Provide a 1-2 max sentences overall summary for the entire text in input using very high rated sentences from TASK 1. Focus more on measurements.
MAXIMIZE the USE of RATED SENTENCES in SUMMARY. MAXIMUM 2 sentences. Third person language.

Text:
\"\"\"
{text}
\"\"\"
"""
    return prompt

In [ ]:
%%time
def extract_structured_info(text):

    response = requests.post(
        "http://localhost:11434/api/generate",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "model": "falcon3:7b",
            "prompt": getsummaryprompt(text),
            "stream": False
        })
    )

    if response.status_code != 200:
        raise Exception(f"Failed to call Model: {response.text}")

    # Extract model output text
    model_response = response.json().get("response", "").strip()

    try:
        # Try parsing structured JSON directly
        structured_info = json.loads(model_response)
    except json.JSONDecodeError:
        print("⚠️ Model did not return valid JSON. Raw output:")
        print(model_response)
        structured_info = None

    return structured_info


In [ ]:
%%time
#t = circulars[0]["body"]
t="Z.-Y. Zheng (SHAO), S.R. Zhu (SHAO), J. X. Wang (USTC), J. E. Rhoads (NASA/GSFC), S. Malhotra (NASA/GSFC), I. G. B. Wold (NASA/GSFC), F. Barrientos (PUC), L. Infante (LCO), W. Hu (TAMU), C. Jiang (SHAO), D. Xu (NAOC), and F. Valdes (NOIRLab) report on behalf of the CIDER collaboration:\n\nWe observed the near-infrared NB1008 narrowband counterpart (Malesani et al., GCN 39727; see also Kennea et al., GCN 39734; Turpin et al., GCN 39739) of the SVOM/ECLAIRs GRB 250314A (Wang et al., GCN 39719), using the Blanco 4m Telescope equipped with the Dark Energy Camera and a small fraction of the CIDER project’s observing time. A sequence of 19 exposures of 540 s each and 4 exposures of 180s each (3.1 hrs in total) was secured in the NB1008 band, with mid time 2025 April 3 (on Apr. 2-4, about 20 days after the GRB).\n\nIn a preliminary reduction, no source is detected at the GRB position down to a 1-sigma upper limiting magnitude NB1008 ~ 23.5 (AB), calibrated with the communication pipeline for DES.\n\nWe also took a 0.35 hrs exposure in the i-band with DECam and no source is detected down to a 1-sigma upper limiting magnitude of i ~ 26.7 (AB), consistent with the high redshift of this GRB. \n\nIf the redshift 7.27 is correct from the previous spectroscopic and photometric confirmation (z ~ 7.27; Malesani et al., GCN 39732; Rakotondrainibe et al., GCN 39743), the NB1008 filter would cover the pure Lya flux of the GRB host galaxy. The NB1008 band‘s 1-sigma upper limit of 23.5 (AB) can be converted to a Lya flux of < 4.2e-17 erg/cm^2/s, and a Lya luminosity of < 3.1e+43 erg/s at z=7.27. Based on the line emission ratio of Lya to Ha in Case B recombination (Osterbrock et al. 1989) and the relation between SFR and the Ha luminosity (Kennicutt 1998), the derived SFR of the GRB host galaxy is < 28 M_sun/yr (1-sigma upper limit). \n"
#t="Camila Angulo (UNAM), Benjamin Schneider (LAM), Rosa L. Becerra (U Roma), Stéphane Basa (UAR Pytheas), William H. Lee (UNAM), Alan M. Watson (UNAM), Fredd Alvarez (UNAM), Jean-Luc Atteia (IRAP), Dalya Akl (AUS), Sarah Antier (OCA), Nathaniel R. Butler (ASU), Damien Dornic (CPPM), Jean-Grégoire Ducoin (CPPM), Francis Fortin (IRAP), Leonardo García García (UNAM), Ramandeep Gill (UNAM), Noémie Globus (UNAM), Kin Ocelotl López (UNAM), Diego López-Cámara (UNAM), Francesco Magnani (CPPM), Enrique Moreno Méndez (UNAM), Margarita Pereyra (UNAM), Ny Avo Rakotondrainibe (LAM) and Antonio de Ugarte Postigo (LAM):\n\nWe imaged the field of the EP250526a (Li et al., GCN Circ. 40550) using the DDRAGO wide-field imager on the COLIBRÍ telescope. We observed from 2025-05-27 03:30 to 5:35 UTC (from 8.0 to 10.1 hours after the trigger and 58 minutes after the notice) and obtained 32 minutes of exposure in g, r and i filters (each one).\n\nThe data were reduced and coadded with the COLIBRÍ pipeline and analysed with STDWeb/STDPipe (Karpov 2025). The photometry was calibrated using nearby stars from the SkyMapper catalog, is in the AB system, and is not corrected for Galactic extinction.\n\nIn the stacked images, we do not detect any new source at the FXT position (Li et al., GCN Circ. 40550) down to the following 3-sigma limit:\n\ng > 23.6\nr > 23.4\ni > 23.0\n\nWe thank the staff of the Observatorio Astronómico Nacional on the Sierra de San Pedro Mártir and the COLIBRÍ and DDRAGO engineering teams.\n\nCOLIBRÍ is an astronomical observatory developed and operated jointly by France (AMU, CNES and CNRS) and Mexico (UNAM and SECIHTI). It is located at the Observatorio Astronómico Nacional on the Sierra de San Pedro Mártir, Baja California, Mexico."
#t="V.Lipunov, E.Gorbovskoy, A.Kuznetsov, K.Zhirkov, I.Panchenko, N.Tiurina, P.Balanutsa, V.Topolev, D.Vlasenko, \nG.Antipov,  A.Sankovich, Yu.Tselik, Ya.Kechin, V.Senik, A.Chasovnikov, K.Labsina, I. Gorbunov (Lomonosov MSU),\nO.Gress, N.Budnev (ISU),\nC.Francile,  F. Podesta, R.Podesta, E. Gonzalez  (Observatorio Astronomico Felix Aguilar (OAFA),\nA. Tlatov, D. Dormidontov (Kislovodsk Solar Station of the Pulkovo Observatory),\nA.Sosnovskij (CrAO),\nA. Gabovich, V.Yurkov (Blagoveschensk Educational StateUniversity),\nD.Buckley (SAAO),\nR.Rebolo (The Instituto de Astrofisica de Canarias),\nL.Carrasco, J.R.Valdes, V.Chavushyan, V.M.Patino Alvarez, J.Martinez,\nA.R.Corella, L.H.Rodriguez (INAOE, Guillermo Haro Astrophysics Observatory) \n\nMASTER-OAFA robotic telescope  (Global MASTER-Net: http://observ.pereplet.ru, Lipunov et al., 2010, Advances in Astronomy, vol. 2010, 30L)  located in Argentina (OAFA observatory of San Juan National University) was pointed to the EP250526a ( EP Team et al., GCN 40550) errorbox  15422 sec after notice time and 42778 sec after trigger time at 2025-05-27 07:21:21 UT, with upper limit up to  18.6 mag. The observations began at zenith distance = 57 deg. The sun  altitude  is -51.0 deg. \n\nThe galactic latitude b = 24 deg., longitude l = 325 deg.\n\n\nReal time updated cover map and OT discovered available here: \nhttps://master.sai.msu.ru/site/master2/observ.php?id=2883070\n\nWe obtain a following upper limits.  \n\nTmid-T0  |      Date Time      |          Site       |             Coord (J2000)          |Filt.| Expt. | Limit| Comment\n_________|_____________________|_____________________|____________________________________|_____|_______|_______|________\n\n   42809 | 2025-05-27 07:21:21 |         MASTER-OAFA | (14h 28m 05.95s , -34d 00m 45.3s) |   C |    60 | 18.6 |        \nFilter C is a clear (unfiltred) band. \n\n\nThe observation and reduction will continue. \nThe message may be cited.\n"
#t="D. Y. Li (NAO, CAS), B.-T. Wang (YNAO, CAS), W. F. Wen (SZTU), J. H. Wu (GZHU), Y. Liu (NAO, CAS) on behalf of the Einstein Probe (EP) team:\n\nWe report on the detection of a fast X-ray transient by the Wide-field X-ray Telescope (WXT) on board the Einstein Probe (EP) mission, designated EP250526a. The transient triggered EP-WXT (ID: 01709176979) at 2025-05-26 19:28:23 (UTC). The WXT position of the source is R.A. = 217.049 deg, DEC = -34.355 deg (J2000) with an uncertainty of 2.9 arcmin in radius (90% C.L. statistical and systematic).\n\nA follow-up observation with the Follow-up X-ray Telescope (FXT) was performed automatically. Within the WXT error circle, an uncatalogued X-ray source was detected at R.A. = 217.0509 deg, DEC = -34.3293 deg (J2000) with an uncertainty of 10 arcsec in radius (90% C.L. statistical and systematic).\n\nFurther information will be updated when the telemetry data is received.\n\nLaunched on January 9, 2024, EP is a space X-ray observatory to monitor the soft X-ray sky with X-ray follow-up capability (Yuan et al. 2022, Handbook of X-ray and Gamma-ray Astrophysics)."
#t="I. Perez-Garcia, E. Fernandez-Garcia, M.D. Caballero-Garcia, R. Sanchez-Ramirez, S. Guziy, S.-Y. Wu, G. Garcia-Segura and A. J. Castro-Tirado (IAA-CSIC, Granada), P. J. Meintjes and H. J. van Heerden (UFS, South Africa), A. Martin-Carrillo and L. Hanlon (UCD, Ireland), M. Gritsevich (Univ. of Helsinki) and C. J. Perez del Pulgar (UMA, Malaga), on behalf of a larger collaboration, report:\n\nFollowing the detection of X-ray transient candidate EP J1350.0-8622 by Einstein Probe (Liu et al., GCNC [40548](https://gcn.nasa.gov/circulars/40548)), the BOOTES-6/DPRT 0.6m robotic telescope at Boyden Observatory in Maselspoort (South Africa) responded to this event starting on May 26, 16:59 UT (~ 17.7 h after trigger). No new source is detected in the WXT region up to magnitude 20.3 using the clear filter and Gaia DR3 Gmag magnitude as reference.\n\nWe thank the staff at Boyden Observatory for their excellent support."
#t="M. J. Liu, D. H. Zhao (NAO, CAS), Y. Wang(PMO, CAS), W. D. Zhang (NAO, CAS) on behalf of the Einstein Probe (EP) team:\n\nWe report on the detection of an X-ray transient candidate at 2025-05-25T23:18:23 (UTC) by the Wide-field X-ray Telescope (WXT) on board Einstein Probe (EP), which we designated EP J1350.0-8622. This source was discovered through on-ground analysis of WXT telemetry data. The WXT position of the source is R.A. = 207.494 deg, DEC = -86.373 deg (J2000) with an uncertainty of 2.6 arcmin in radius (90% C.L. statistical and systematic). The derived unabsorbed 0.5-4 keV flux by an absorbed power-law model is 6.8(+9.6/-6.8) x 10^-12 erg/cm^2/s with the column density fixed at the Galactic value of 8.2 x 10^20 cm^-2 and photon index fixed at 2.\n\nWithin the WXT error circle, an X-ray source 1eRASS J135031.1-862323 was detected by eROSITA with a flux of 4.9 x 10^-14 erg/cm2/s (0.6-2.3 keV). A Gaia source Gaia DR3 5766213157904565376, likely a quasar, is 8.155 arcsec away from the eROSITA source.\n\n\nLaunched on January 9, 2024, EP is a space X-ray observatory to monitor the soft X-ray sky with X-ray follow-up capability (Yuan et al. 2022, Handbook of X-ray and Gamma-ray Astrophysics).\n"
#t="Chao WU (NAOC), Zhe Kang (CHO), Liping Xin(NAOC), Xuhui Han(NAOC), Pinpin Zhang (NAOC), Xiaomeng Lu (NAOC), Zhenwei Li (CHO), You Lv (CHO), Ruosong Zhang (NAOC), Yujie Xiao (NAOC), Yulei, Qiu(NAOC), Jing Wang (NAOC), Jinsong Deng (NAOC), Lei Huang (NAOC), Jianyan Wei (NAOC) report on behalf of the SVOM/C-GFT team:\n\n\nWe observed the field of the X-ray transient WXT trigger 01709176712 from the Einstein Probe (Yang et al., GCN 40537) with LATIOS on SVOM/C-GFT. Observations started at 2025-05-23T12:52:03, ~7.54 mins after the trigger.  \n\nA series of g, r, and i-band images were obtained over ~2.58 hr. Preliminary analysis shows that the flare star reported by Yang et al. (GCN 40537) and Neill et al. (GCN 40542) exhibited a g-band magnitude decline from 11.36 mag to 12.95 mag, with a color variation (g−i) of~ 0.73 mag (from −0.24 to +0.49) between 11 and 162 minutes post trigger.\n\n\n\nMagnitudes were calibrated with nearby Pan-STARRS1 stars and were not corrected for dust extinction.\n\n\n\nWe thank the observation assistants Bowen Li and Shuai Liu at Jilin observatory for their excellent support.\n\n\n\nThe Chinese Ground Follow-up Telescope (C-GFT) for the SVOM mission is located at Jilin Station, Changchun Observatory, National Astronomical Observatories, CAS. It features two instruments: (1) CATCH at the Cassegrain focus with a 21 arcsec ×21 arcsec FOV for simultaneous g/r/i-band imaging, and (2) LATIOS, a 4k×4k CMOS camera at the prime focus with a 1.28 deg × 1.28 deg FOV that images in g, r, and i bands via filter switching.\n"
#t="M. J. Liu, D. H. Zhao (NAO, CAS), Y. Wang(PMO, CAS), W. D. Zhang (NAO, CAS) on behalf of the Einstein Probe (EP) team:\n\nWe report on the detection of an X-ray transient candidate at 2025-05-25T23:18:23 (UTC) by the Wide-field X-ray Telescope (WXT) on board Einstein Probe (EP), which we designated EP J1350.0-8622. This source was discovered through on-ground analysis of WXT telemetry data. The WXT position of the source is R.A. = 207.494 deg, DEC = -86.373 deg (J2000) with an uncertainty of 2.6 arcmin in radius (90% C.L. statistical and systematic). The derived unabsorbed 0.5-4 keV flux by an absorbed power-law model is 6.8(+9.6/-6.8) x 10^-12 erg/cm^2/s with the column density fixed at the Galactic value of 8.2 x 10^20 cm^-2 and photon index fixed at 2.\n\nWithin the WXT error circle, an X-ray source 1eRASS J135031.1-862323 was detected by eROSITA with a flux of 4.9 x 10^-14 erg/cm2/s (0.6-2.3 keV). A Gaia source Gaia DR3 5766213157904565376, likely a quasar, is 8.155 arcsec away from the eROSITA source.\n\n\nLaunched on January 9, 2024, EP is a space X-ray observatory to monitor the soft X-ray sky with X-ray follow-up capability (Yuan et al. 2022, Handbook of X-ray and Gamma-ray Astrophysics).\n"
#t = "The 300V grism with no order-blocking filter was adopted, covering the wavelength range 3300-9600 AA (with potential second-order contamination redward of 6600 AA). Observations started on 2025 April 4 at 02:15:01 UT (10.99 hr after the burst). Two exposures of 600 s each were obtained.\n\nFrom a 60 s acquisition image obtained on 2025 April 4 at 02:01:37 UT (10.76 hr after the burst), we measure R ~ 21.1 +/- 0.2 mag (AB) calibrated against nearby Pan-STARRS objects.\n\nFrom a preliminary reduction of the spectrum, continuum is detected over the wavelength range 3600-9600 AA. While the spectrum is noisy blueward of 3720 AA, we can set an upper limit to the redshift z < 2.06 due to the lack of Lyman forest. Four clear absorption features are apparent in clean regions of the spectrum, three of which match Si IV 1393, Si IV 1402 and the (blended) C IV doublet 1548,1550 at a common redshift z = 1.847. The fourth line seems marginally split and could be an intervening C IV doublet at z = 1.763, although no other features are seen to confirm this value.\n\nThe observed absorption system is somewhat peculiar (but not unprecedented), as some typically strong low-ionization features common in long GRB spectra (de Ugarte Postigo et al. 2012, doi:10.1051/0004-6361/201219894) are either non-detected or only marginally detected. This indicates a high-ionization environment, which has been previously linked to systems with low H I column density (e.g. Jakobsson et al. 2006, doi:10.1051/0004-6361:20066405; Thoene et al. 2011, doi:10.1111/j.1365-2966.2011.18408.x; Vielfaure et al. 2020, doi:10.1051/0004-6361/202038316). We thus consider z = 1.847 a viable possibility, and the most likely redshift of GRB 250403A.\n\nWe acknowledge excellent support from the ESO observing staff in Paranal, in particular Matias Jones, Claudia Paladini, Jesus Corral-Santana, and Cecilia Bustos."
#t="Real time updated cover map and OT discovered available here: \nhttps://master.sai.msu.ru/site/master2/observ.php?id=2866536\n\nWe obtain a following upper limits.  \n\nTmid-T0  |      Date Time      |          Site       |             Coord (J2000)          |Filt.| Expt. | Limit| Comment\n_________|_____________________|_____________________|____________________________________|_____|_______|_______|________\n\n      70 | 2025-05-09 22:34:33 |         MASTER-OAFA | (03h 18m 00.28s , -38d 23m 19.1s) |   C |    10 | 14.1 |        \n     104 | 2025-05-09 22:35:03 |         MASTER-OAFA | (03h 18m 00.24s , -38d 23m 19.1s) |   C |    20 | 14.5 |        \n     138 | 2025-05-09 22:35:42 |         MASTER-OAFA | (03h 18m 00.14s , -38d 23m 18.9s) |   C |    10 | 14.4 |        \n     174 | 2025-05-09 22:36:18 |         MASTER-OAFA | (03h 18m 00.11s , -38d 23m 18.5s) |   C |    10 | 14.4 |        \n     215 | 2025-05-09 22:36:53 |         MASTER-OAFA | (03h 18m 00.10s , -38d 23m 18.6s) |   C |    20 | 14.7 |        \n     265 | 2025-05-09 22:37:39 |         MASTER-OAFA | (03h 18m 00.12s , -38d 23m 19.3s) |   C |    30 | 14.8 |        \n     865 | 2025-05-09 22:46:44 |         MASTER-OAFA | (03h 18m 00.32s , -38d 23m 28.0s) |   C |   140 | 14.8 |        \n    1041 | 2025-05-09 22:49:30 |         MASTER-OAFA | (03h 18m 00.52s , -38d 23m 31.2s) |   C |   160 | 16.3 |        \n    1227 | 2025-05-09 22:52:36 |         MASTER-OAFA | (03h 18m 00.73s , -38d 23m 34.3s) |   C |   160 | 16.5 |        \n    1414 | 2025-05-09 22:55:42 |         MASTER-OAFA | (03h 18m 01.03s , -38d 23m 38.0s) |   C |   160 | 17.0 |        \n    1424 | 2025-05-09 22:55:43 |         MASTER-OAFA | (03h 06m 24.43s , -38d 43m 44.4s) |   C |   180 | 18.2 |        \n    1600 | 2025-05-09 22:58:49 |         MASTER-OAFA | (03h 18m 01.35s , -38d 23m 42.3s) |   C |   160 | 17.1 |        \n    1610 | 2025-05-09 22:58:49 |         MASTER-OAFA | (03h 06m 24.67s , -38d 43m 47.0s) |   C |   180 | 18.4 |        \n    1786 | 2025-05-09 23:01:55 |         MASTER-OAFA | (03h 18m 01.74s , -38d 23m 46.7s) |   C |   160 | 17.1 |        \n    1796 | 2025-05-09 23:01:55 |         MASTER-OAFA | (03h 06m 24.96s , -38d 43m 50.0s) |   C |   180 | 18.3 |        \n    1972 | 2025-05-09 23:05:01 |         MASTER-OAFA | (03h 18m 02.05s , -38d 23m 50.9s) |   C |   160 | 17.1 |        \n    1983 | 2025-05-09 23:05:02 |         MASTER-OAFA | (03h 06m 25.27s , -38d 43m 53.2s) |   C |   180 | 18.2 |        \n    2159 | 2025-05-09 23:08:08 |         MASTER-OAFA | (03h 18m 02.30s , -38d 23m 55.2s) |   C |   160 | 17.1 |        \n    2169 | 2025-05-09 23:08:08 |         MASTER-OAFA | (03h 06m 25.58s , -38d 43m 56.5s) |   C |   180 | 18.1 |        \n    2356 | 2025-05-09 23:11:14 |         MASTER-OAFA | (03h 06m 25.97s , -38d 44m 00.1s) |   C |   180 | 18.0 |        \n    2531 | 2025-05-09 23:14:20 |         MASTER-OAFA | (03h 18m 02.98s , -38d 24m 03.8s) |   C |   160 | 17.1 |        \n    2542 | 2025-05-09 23:14:20 |         MASTER-OAFA | (03h 06m 26.35s , -38d 44m 03.6s) |   C |   180 | 18.0 |        \n    2718 | 2025-05-09 23:17:26 |         MASTER-OAFA | (03h 18m 03.37s , -38d 24m 09.0s) |   C |   160 | 17.2 |        \n    2728 | 2025-05-09 23:17:27 |         MASTER-OAFA | (03h 06m 26.66s , -38d 44m 06.7s) |   C |   180 | 18.0 |        \n    2904 | 2025-05-09 23:20:33 |         MASTER-OAFA | (03h 18m 03.73s , -38d 24m 14.1s) |   C |   160 | 17.1 |        \n    2914 | 2025-05-09 23:20:33 |         MASTER-OAFA | (03h 06m 26.94s , -38d 44m 08.8s) |   C |   180 | 17.6 |        \n    3090 | 2025-05-09 23:23:39 |         MASTER-OAFA | (03h 18m 04.07s , -38d 24m 19.9s) |   C |   160 | 16.9 |        \n    3100 | 2025-05-09 23:23:39 |         MASTER-OAFA | (03h 06m 27.22s , -38d 44m 12.4s) |   C |   180 | 17.4 |       "
output = extract_structured_info(t)

In [ ]:
print(output)